# Session 3: Further Pandas

#### 1. NYC buildings - Load and inspect a noisy dataset
We are using the PLUTO (Primary Land Use Tax Lot Output) database, which lists every building in New York City.
Source is [NYC Open Data Portal](https://www1.nyc.gov/site/planning/data-maps/open-data/dwn-pluto-mappluto.page). Check the [data dictionary](https://www1.nyc.gov/assets/planning/download/pdf/data-maps/open-data/pluto_datadictionary.pdf?r=18v2beta).

In [ ]:
import numpy as np
import pandas as pd
import pylab as plt
%pylab inline

In [ ]:
# Create a dataframe using read_csv()

url = 'https://github.com/worldbank/Python-for-Data-Science/raw/master/Spring%202019%208-week%20course/week%203/pluto_shorter.csv'

df = pd.read_csv(url)

In [ ]:
# how big is it?

df.shape    # bonus: can you print 'loaded dataset with x rows and y columns'?

In [ ]:
# inspect with .head()

df.head()

#### 2. Clean up the dataset
Also known as 'data munging'

In [ ]:
# List the columns

df.columns

In [ ]:
# make a list of desired columns; discard the rest.

my_cols = ['borough','numfloors','yearbuilt', 'landuse', 'zipcode', 'unitstotal', 'assesstot','policeprct']
df = df[my_cols]

In [ ]:
# isnull() checks for missing values

df.borough.isnull()

In [ ]:
# what data structure would be helpful to rename columns? (think 'old_name' : 'new_name')

df.rename(columns = {'zipcode': 'zip', 'yearbuilt': 'year_built', 'unitstotal': 'housing_units', 'assesstotal': 'assessed_value_USD'},
         inplace = True)

In [ ]:
# each Series has a data type

In [ ]:
df.dtypes

Memo: `police_precint` would be better as an int.

### 3. Exploratory visualization
Pandas allows quick, iterative plotting to explore the properties of your data - helpful to diagnose data quality issues.

A histogram plots the frequency of each unique value. Let's plot one for year_built. Any problem?

In [ ]:
# .hist() plots a histogram for all values in a Series.

df.year_built.hist()

In [ ]:
# pause to get help on the .hist() method

df.hist?

In [ ]:
# Presumably few NYC buildings were constructed during the early days of the Roman Empire.
# Set year_built to np.nan for such values.

df.year_built[df.year_built < 1000] = np.nan

In [ ]:
# histogram should make sense without the erroneous values

df.year_built.hist()

#### 4. Handle missing values
How many missing values?

In [ ]:
# isnull() evaluates each item, returning True if NaN and False otherwise

df.isnull().sum()

 What steps to take, eg:
* drop rows with missing values
* deduce the missing values (eg. ZIP from coordinates)
* fill NaNs using mean, median or a custom strategy

In [ ]:
# find methods to address missing values


In [ ]:
# check how a method works and what parameters it needs

df.fillna?


In [ ]:
# For now, drop all rows with NaNs

print('original shape: ', df.shape)

df.dropna(inplace=True)

print('new shape: ', df.shape)

#### 5. Compute summary statistics

In [ ]:
# Use methods like max, min and mean() on a column

print("Max floors: ", df.numfloors.max())
print("Mean assessed value: {:.0f}".format(df.assesstot.mean()))

In [ ]:
# Use plotting to explore relationships in the data

df.plot(x = 'year_built', y = 'numfloors', kind = 'scatter', title = 'NYC buildings: Year built versus number of floors',figsize = (10,6));

## Second example - unemployment data
*Objectives: Merge datasets, fill missing data through interpolation, use time series*

In [ ]:
# Load a dataset of Eurostat unemployment data 

unemployment = pd.read_csv('https://raw.githubusercontent.com/worldbank/Python-for-Data-Science/master/Spring%202019%208-week%20course/week%205/data/country_total.csv')

In [ ]:
# Inspect with the .head() method

unemployment.head()

In [ ]:
# the .shape attribute gives a tuple with row and column numbers

unemployment.shape

You may notice that the month column actually shows the year and month. Let's rename it.

In [ ]:
# Pass a dictionary to the .rename() method, specifying the column to rename (the dictionary key) and the new name to use (the dictionary value).

unemployment.rename(columns = {'month': 'year_month'}, inplace = True)

**Data munging**

This column is not easy to work with:

In [ ]:
unemployment['year_month'].head()

Let's separate year and month into their own columns:

In [ ]:
# Step by step: first make the column contain strings

unemployment.year_month = unemployment.year_month.astype(str)

In [ ]:
# Let's check what an example cell looks like


In [ ]:
# What is this code structure called? Practice it as homework.

[string.split('.')[0] for string in unemployment.year_month][:5]

In [ ]:
# Reassign to new column

unemployment['year'] = [string.split('.')[0] for string in unemployment.year_month]

In [ ]:
unemployment['month'] = [string.split('.')[1] for string in unemployment.year_month]

It would be better to see the full names of the countries. Let's load another csv that has these.

In [ ]:
countries_url = 'https://raw.githubusercontent.com/dlab-berkeley/introduction-to-pandas/master/data/countries.csv'
countries = pd.read_csv(countries_url)

In [ ]:
countries.head()

**Merging dataframes**

Now, let's merge the two dfs, to get full country name and country_group.

In [ ]:
country_names = countries[['country','name_en','country_group']]

In [ ]:
unemployment = pd.merge(unemployment, country_names, on='country')

Check the full [documentation](http://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging) on merging.

**Value counts and sorting**

What's the state of unemployment data for these countries? Do some countries report more frequently?

In [ ]:
# these options mean not seasonally adjusted, seasonally adjusted, trend cycle

unemployment.seasonality.unique()

In [ ]:
# .value_counts() return a Series containing unique values as its index and
# frequencies as its values, in descending order.

unemployment.name_en.value_counts().sort_values(ascending = True)[:15]

In [ ]:
print("Dataset runs from {} to {}".format(unemployment.year.min(), unemployment.year.max()))

**View subsets of the data based on conditions**

You can subset the data using the syntax below.

For now, just learn and adapt this syntax. As homework, look up Boolean indexing in the McKinney book and see why this works.


In [ ]:
unemployment[unemployment.name_en == 'Italy'].head(2)

In [ ]:
unemployment[unemployment.unemployment_rate < 10].head(2)

#### Summarize values with `groupby()`
A `groupby()` operation carries out some combination of splitting the data, performing an operation, and combining the results:

* state the Series you want to group by.
* append the operation you want to perform such as .mean(), .sum() or .count()

In [ ]:
unemployment.head(2)

In [ ]:
unemployment_small = unemployment[['unemployment_rate','name_en']]

unemployment_small.head()

In [ ]:
unemployment_small.groupby('name_en',sort=True).mean()

In [ ]:
unemployment_small.groupby('name_en',sort=True).mean().sort_values(by = 'unemployment_rate', ascending=False)[:10]

**Drop or fill missing values**

It's not surprising that different statistical agencies collect unemployment stats at different frequencies. But to compare say Spain and Portugal, we may wish to fill the missing value gaps. Pandas has several options, a simple one being ffill (forward fill).

Let's look at two neighbors with high unemployment: Latvia and Lithuania

In [ ]:
from dateutil.parser import parse

In [ ]:
unemployment.head(2)

#### Add a datetime index

In [ ]:
# recombine the year and month columns

unemployment['year_month'] = unemployment['year'] + ":"+ unemployment['month']

In [ ]:
# specify a format to parse datetimes, based on these: %d/%m/%Y”

unemployment['date_time'] = pd.to_datetime(unemployment.year_month, format='%Y:%m')

In [ ]:
# make it the df's index

unemployment.set_index('date_time',drop=True, inplace = True)

In [ ]:
start_date = parse('1995-01-01')
end_date = parse('2000-01-01')

In [ ]:
unemployment[unemployment.index > start_date][:5]

In [ ]:
f,ax = plt.subplots(figsize = (8,5))

unemployment.unemployment_rate[unemployment.name_en == 'Latvia'].plot(ax = ax)
unemployment.unemployment_rate[unemployment.name_en == 'Lithuania'].plot(ax = ax)

ax.set(title="Unemployment: Lithuania and Latvia")